- consider only cases where output size is larger than input
- add new dsls

In [1]:
import os
import json
import random
import inspect
import itertools
import numpy as np
import pandas as pd
from pathlib import Path
from skimage import measure
from matplotlib import colors
import matplotlib.pyplot as plt
from itertools import permutations
from numpy.lib.stride_tricks import as_strided

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'
training_tasks = sorted(os.listdir(training_path))
evaluation_tasks = sorted(os.listdir(evaluation_path))
test_tasks = sorted(os.listdir(test_path))

num2color = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown"]
color2num = {c: n for n, c in enumerate(num2color)}

In [2]:
def preprocess(task, index): # modify mistakes in data
    # 025d127b
    if index == "025d127b.json":
        for i in range(9, 12):
            for j in range(3, 8):
                task['train'][0]['output'][i][j] = 0
        for i in range(7, 10):
            for j in range(3, 6):
                task['train'][0]['output'][i][j] = 2
        task['train'][0]['output'][8][4] = 0
    # ef135b50
    elif index == "ef135b50.json":
        task['test'][0]['output'][6][4] = 9
    # bd14c3bf
    elif index == "bd14c3bf.json":
        for i in range(3):
            for j in range(5):
                if task['test'][0]['input'][i][j] == 1:
                    task['test'][0]['input'][i][j] = 2
    # a8610ef7
    elif index == "a8610ef7.json":
        for i in range(6):
            for j in range(6):
                if task['test'][0]['output'][i][j] == 8:
                    task['test'][0]['output'][i][j] = 5
        task['train'][3]['input'][0][1] = 2
        task['train'][3]['input'][5][1] = 2
    # 54db823b
    elif index == "54db823b.json":
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 9
    # e5062a87
    elif index == "e5062a87.json":
        for j in range(3, 7):
            task['train'][1]['output'][1][j] = 2
    # 1b60fb0c
    elif index == "1b60fb0c.json":
        task['train'][1]['output'][8][8] = 0
        task['train'][1]['output'][8][9] = 0
    # 82819916
    elif index == "82819916.json":
        task['train'][0]['output'][4][5] = 4
    # fea12743
    elif index == "fea12743.json":
        for i in range(11, 16):
            for j in range(6):
                if task['train'][0]['output'][i][j] == 2:
                    task['train'][0]['output'][i][j] = 8
    # 42a50994
    elif index == "42a50994.json":
        task['train'][0]['output'][1][0] = 8
        task['train'][0]['output'][0][1] = 8
    # f8be4b64
    elif index == "f8be4b64.json":
        for j in range(19):
            if task['test'][0]['output'][12][j] == 0:
                task['test'][0]['output'][12][j] = 1
        task['test'][0]['output'][12][8] = 0
    # d511f180
    elif index == "d511f180.json":
        task['train'][1]['output'][2][2] = 9
    # 10fcaaa3
    elif index == "10fcaaa3.json":
        task['train'][1]['output'][4][7] = 8
    # cbded52d
    elif index == "cbded52d.json":
        task['train'][0]['input'][4][6] = 1
    # 11852cab
    elif index == "11852cab.json":
        task['train'][0]['input'][1][2] = 3
    # 868de0fa
    elif index == "868de0fa.json":
        for j in range(2, 9):
            task['train'][2]['input'][9][j] = 0
            task['train'][2]['input'][10][j] = 1
            task['train'][2]['input'][15][j] = 0
            task['train'][2]['input'][16][j] = 1
        task['train'][2]['input'][15][2] = 1
        task['train'][2]['input'][15][8] = 1
    # 6d58a25d
    elif index == "6d58a25d.json":
        task['train'][0]['output'][10][0] = 0
        task['train'][2]['output'][6][13] = 4
    # a9f96cdd
    elif index == "a9f96cdd.json":
        task['train'][3]['output'][1][3] = 0
    # 48131b3c
    elif index == "48131b3c.json":
        task['train'][2]['output'][4][4] = 0
    # 150deff5
    elif index == "150deff5.json":
        aux = task['train'][2]['output'].copy()
        task['train'][2]['output'] = task['train'][2]['input'].copy()
        task['train'][2]['input'] = aux
    # 17cae0c1
    elif index == "17cae0c1.json":
        for i in range(3):
            for j in range(3, 6):
                task['test'][0]['output'][i][j] = 9
    # e48d4e1a
    elif index == "e48d4e1a.json":
        task['train'][3]['input'][0][9] = 5
        task['train'][3]['output'][0][9] = 0
    # 8fbca751
    elif index == "8fbca751.json":
        task['train'][1]['output'][1][3] = 2
        task['train'][1]['output'][2][3] = 8
    # 4938f0c2
    elif index == "4938f0c2.json":
        for i in range(12):
            for j in range(6,13):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
        for i in range(5,11):
            for j in range(7):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
    # 9aec4887
    elif index == "9aec4887.json":
        task['train'][0]['output'][1][4] = 8
    # b0f4d537
    elif index == "b0f4d537.json":
        for i in range(9):
            task['train'][0]['output'][i][3] = 0
            task['train'][0]['output'][i][4] = 1
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 3
        task['train'][0]['output'][5][3] = 2
    # aa300dc3
    elif index == "aa300dc3.json":
        task['train'][1]['input'][1][7] = 5
        task['train'][1]['output'][1][7] = 5
        task['train'][1]['input'][8][2] = 5
        task['train'][1]['output'][8][2] = 5
    # ad7e01d0
    elif index == "ad7e01d0.json":
        task['train'][0]['output'][6][7] = 0
    # a8610ef7
    elif index == "a8610ef7.json":
        task['train'][3]['input'][0][1] = 0
        task['train'][3]['input'][5][1] = 0
        task['train'][3]['output'][0][1] = 0
        task['train'][3]['output'][5][1] = 0
    # 97239e3d
    elif index == "97239e3d.json":
        task['test'][0]['input'][14][6] = 0
        task['test'][0]['input'][14][10] = 0
    # d687bc17
    elif index == "d687bc17.json":
        task['train'][2]['output'][7][1] = 4
    return task

In [3]:
def color_check(color_list):
    tmp = color_list[0]
    for i in range(1,len(color_list)):
        if set(tmp) != set(color_list[i]):
            return False
    return True

def create_df(folder_path):
    task_names_list = sorted(os.listdir(folder_path))
    task_list = []
    for task_name in task_names_list: 
        task_file = str(folder_path / task_name)
        with open(task_file, 'r') as f:
            task = json.load(f)
            if "test" not in str(folder_path):
                task = preprocess(task, task_name)
            task_list.append(task)
    
    df = pd.DataFrame()
    df['task_name'] = task_names_list
    df['task'] = task_list
    df['number_of_train_pairs'] = df['task'].apply(lambda x: len(x['train']))
    df['number_of_test_pairs'] = df['task'].apply(lambda x: len(x['test']))
    
    # Compare image sizes
    df['inputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['input']) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['input'][0]) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_shape'] = df['inputs_all_have_same_height'] * df['inputs_all_have_same_width']
    df['input_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'])
                     if (len(set([len(example['input']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['input_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) for example in task['train']])) == 1)
                     else np.nan
    )
    df['outputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['output']) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['output'][0]) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_shape'] = df['outputs_all_have_same_height'] * df['outputs_all_have_same_width']
    df['output_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'])
                     if (len(set([len(example['output']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['output_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0])
                     if (len(set([len(example['output'][0]) for example in task['train']])) == 1)
                     else np.nan
    )  
    df['in_each_pair_shape_doesnt_change'] = df['task'].apply(
        lambda task: np.prod([int(len(example['input'][0])==len(example['output'][0])
                                  and len(example['input'])==len(example['output'])
                                 ) for example in task['train']
                            ])
    )
    df['in_each_pair_shape_ratio_is_the_same'] = df['task'].apply(
        lambda task: (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1) * (
                      len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
    )
    df['o/i_height_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output']) / len(task['train'][0]['input'])
                     if (len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    df['o/i_width_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0]) / len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    
    # my idea ---------
    df["same_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input'])))== sum(sum(np.array(example['output'])))) for example in task['train']]))
    
    df["same_color_sum_in_edge"] = df['task'].apply(lambda task: 
                        np.all([int(sum(np.array(example['input'])[0,:]) +sum(np.array(example['input'])[:,0]) + 
                                    sum(np.array(example['input'])[-1,:]) +sum(np.array(example['input'])[:,-1])
                                    == 
                                    sum(np.array(example['output'])[0,:]) +sum(np.array(example['output'])[:,0]) + 
                                    sum(np.array(example['output'])[-1,:]) +sum(np.array(example['output'])[:,-1])) for example in task['train']]))
    
    df["io_color_kind_diff"] = df['task'].apply(lambda task: [len(np.unique(np.array(example['input']))) - len(np.unique(np.array(example['output']))) for example in task['train']])
    df["io_color_kind_diff_constant"] = df['io_color_kind_diff'].apply(lambda task: np.unique(np.array(task))[0] if len(np.unique(np.array(task)))==1 else -1)
    df["output_not_include_0"] = df['task'].apply(lambda task: np.all([np.all(np.array(example['output']) > 0) for example in task['train']]))
    df["increase_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) < sum(sum(np.array(example['output'])))) for example in task['train']]))
    df["decrease_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) > sum(sum(np.array(example['output'])))) for example in task['train']]))
    # **
    df["input_color_change_or_not"] = df['task'].apply(lambda task: color_check([list(np.unique(np.array(example['input']))) for example in task["train"]] ))
    # **
    df['color_kind_increase'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) < len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['color_kind_decrease'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) > len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['smaller_output'] = df['task'].apply(
        lambda task: np.all([(np.array(example['input']).shape[0] >= np.array(example['output']).shape[0] and 
                             np.array(example['input']).shape[1] >= np.array(example['output']).shape[1] and 
                             (np.array(example['input']).shape[0] > np.array(example['output']).shape[0] or 
                             np.array(example['input']).shape[1] > np.array(example['output']).shape[1])) for example in task['train']]))
    df['larger_output'] = df['task'].apply(
        lambda task: np.all([(np.array(example['input']).shape[0] <= np.array(example['output']).shape[0] and 
                             np.array(example['input']).shape[1] <= np.array(example['output']).shape[1] and 
                             (np.array(example['input']).shape[0] < np.array(example['output']).shape[0] or 
                             np.array(example['input']).shape[1] < np.array(example['output']).shape[1])) for example in task['train']]))
    return df

training_descriptive_df = create_df(training_path)
evaluation_descriptive_df = create_df(evaluation_path)
test_descriptive_df = create_df(test_path)

In [4]:
def classification(row):
    # same shape and color doesn't change in input and color kind decrease
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_decrease==1:
        return 1
    # same shape and color doesn't change in input and color kind increase
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==1:
        return 2
    # same shape and color doesn't change in input and color kind same
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 3
    # same shape and decrease color sum → xgboost
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_decrease==1:
        return 4
    # different shape and decrease color sum
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==1:
        return 5
    # different shape and increase color sum
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 6
    # otherwise
    elif row["smaller_output"] == 1 and row["outputs_all_have_same_shape"] == 1:
        return 7
    elif row["larger_output"] == 1 and row["outputs_all_have_same_shape"] == 1:
        return 8
    else:
        return 9

training_descriptive_df["class"] = training_descriptive_df.apply(lambda x: classification(x), axis=1)
evaluation_descriptive_df["class"] = evaluation_descriptive_df.apply(lambda x: classification(x), axis=1)
test_descriptive_df["class"] = test_descriptive_df.apply(lambda x: classification(x), axis=1)

# Domain Specific Language (DSL)

We will build a domain specific language specialized on processing list of images. To allow easy chaining of keyword from this language together, each *function* provided by this language will be take one or more images and transform it to none, one or more. The final result of our program will then be a list of images.

The DSL is so constituted by a collection of functions of type `np.array -> [np.array]` and `[np.array] -> [np.array]`.

The first kind of function take an image, and produce a list of images (for example, the image split by different colors). The second type of function take a list of images and produce a new list (for exemple, intersect).
[](http://)

In [5]:
def neighbours(cur_row, cur_col, nrows, ncols): # function for pickup objects function
    if cur_row==0: top = -1
    else: top = [cur_row-1,cur_col]
    if cur_row==nrows-1: bottom = -1
    else: bottom = [cur_row+1, cur_col]
    if cur_col==0: left = -1
    else: left = [cur_row,cur_col-1]
    if cur_col==ncols-1: right = -1
    else: right = [cur_row, cur_col+1]
    if cur_row == 0 or cur_col == ncols-1: tr = -1
    else: tr = [cur_row-1, cur_col+1]
    if cur_row == 0 or cur_col == 0: tl = -1
    else: tl = [cur_row-1, cur_col-1]
    if cur_row == nrows-1 or cur_col == ncols-1: br = -1
    else: br = [cur_row+1, cur_col+1]
    if cur_row == nrows-1 or cur_col == 0: bl = -1
    else: bl = [cur_row+1, cur_col-1]
    ans = []
    for i in [top, bottom, left, right, tr, tl, br, bl]:
        if i != -1:
            ans.append(i)
    return ans

def make_group(loc, h, w): # function for pickup objects function
    ans = [loc[0].tolist()]
    check_list = [loc[0].tolist()]
    remain = loc[1:].tolist()
    while True:
        check_ele = check_list[0]
        check_list.remove(check_ele)
        neigh = neighbours(check_ele[0], check_ele[1], h, w)
        for i in neigh:
            if i in remain:
                ans.append(i)
                check_list.append(i)
                remain.remove(i)
        if len(check_list) == 0:
            break
    return ans, np.array(remain)

def find_inside(points): # general function
    x_s = np.unique(np.array(points)[:,0]).tolist()
    inside = []
    for i in x_s:
        indexes = np.where(np.array(points)[:,0]==i)[0].tolist()
        y_s = [points[j][1] for j in indexes]
        if max(y_s) - min(y_s) >1:
            for k in range(min(y_s)+1,max(y_s)):
                inside.append([i,k])
    return inside

def pickup_gray_area(pixmap):
    try:
        loc = np.argwhere(pixmap == 5)
        h, w = pixmap.shape
        count = 0
        groups = [] # pick up coordinates
        ans = [] # pick up objects
        ans2 = []
        if len(loc) > 0:
            while True:
                if count == 0:
                    tmp, remain = make_group(loc, h, w)
                else:
                    tmp, remain = make_group(remain, h, w)
                groups.append(tmp)
                count += 1
                if len(remain) == 0:
                    break
            for i in groups:    
                x_min = np.min(np.array(i)[:,0]) # x coordinate np.array(i)[:,0]
                x_max = np.max(np.array(i)[:,0]) # x coordinate
                y_min = np.min(np.array(i)[:,1]) # y coordinate np.array(i)[:,1]
                y_max = np.max(np.array(i)[:,1]) # y coordinate
                tmp = [x_min,x_max+1, y_min,y_max+1]
                tmp_area = pixmap[x_min:x_max+1, y_min:y_max+1]
                ans.append(tmp)
                ans2.append(tmp_area)
            return ans, ans2
        else:
            return []
    except:
        return []


# for tile and symmetry -------
def in_out_diff(t_in, t_out):
    x_in, y_in = t_in.shape
    x_out, y_out = t_out.shape
    diff = np.zeros((max(x_in, x_out), max(y_in, y_out)))
    diff[:x_in, :y_in] -= t_in
    diff[:x_out, :y_out] += t_out
    return diff

def check_symmetric(a):
    try:
        sym = 1
        if np.array_equal(a, a.T):
            sym *= 2 #Check main diagonal symmetric (top left to bottom right)
        if np.array_equal(a, np.flip(a).T):
            sym *= 3 #Check antidiagonal symmetric (top right to bottom left)
        if np.array_equal(a, np.flipud(a)):
            sym *= 5 # Check horizontal symmetric of array
        if np.array_equal(a, np.fliplr(a)):
            sym *= 7 # Check vertical symmetric of array
        return sym
    except:
        return 0
    
def bbox(a):
    try:
        r = np.any(a, axis=1)
        c = np.any(a, axis=0)
        rmin, rmax = np.where(r)[0][[0, -1]]
        cmin, cmax = np.where(c)[0][[0, -1]]
        return rmin, rmax, cmin, cmax
    except:
        return 0,a.shape[0],0,a.shape[1]
    
def cmask(t_in):
    cmin = 999
    cm = 0
    for c in range(10):
        t = t_in.copy().astype('int8')
        t[t==c],t[t>0],t[t<0]=-1,0,1
        b = bbox(t)
        a = (b[1]-b[0])*(b[3]-b[2])
        s = (t[b[0]:b[1],b[2]:b[3]]).sum()
        if a>2 and a<cmin and s==a:
            cmin=a
            cm=c
    return cm

def mask_rect(a):
    r,c = a.shape
    m = a.copy().astype('uint8')
    for i in range(r-1):
        for j in range(c-1):
            if m[i,j]==m[i+1,j]==m[i,j+1]==m[i+1,j+1]>=1:m[i,j]=2
            if m[i,j]==m[i+1,j]==1 and m[i,j-1]==2:m[i,j]=2
            if m[i,j]==m[i,j+1]==1 and m[i-1,j]==2:m[i,j]=2
            if m[i,j]==1 and m[i-1,j]==m[i,j-1]==2:m[i,j]=2
    m[m==1]=0
    return (m==2)

def crop_min(t_in):
    try:
        b = np.bincount(t_in.flatten(),minlength=10)
        c = int(np.where(b==np.min(b[np.nonzero(b)]))[0])
        coords = np.argwhere(t_in==c)
        x_min, y_min = coords.min(axis=0)
        x_max, y_max = coords.max(axis=0)
        return t_in[x_min:x_max+1, y_min:y_max+1]
    except:
        return t_in
    
def call_pred_train(t_in, t_out, pred_func):
    try:
        feat = {}
        feat['s_out'] = t_out.shape
        if t_out.shape==t_in.shape:
            diff = in_out_diff(t_in,t_out)
            feat['diff'] = diff
            feat['cm'] = t_in[diff!=0].max()
        else:
            feat['diff'] = (t_in.shape[0]-t_out.shape[0],t_in.shape[1]-t_out.shape[1])
            feat['cm'] = cmask(t_in)
        feat['sym'] = check_symmetric(t_out)
        args = inspect.getfullargspec(pred_func).args
        if len(args)==1:
            return pred_func(t_in)
        elif len(args)==2:
            t_pred = pred_func(t_in,feat[args[1]])    
        elif len(args)==3:
            t_pred = pred_func(t_in,feat[args[1]],feat[args[2]])
        feat['sizeok'] = len(t_out)==len(t_pred)
        t_pred = np.resize(t_pred,t_out.shape)
        return t_pred, feat
    except:
        return t_in, feat

def call_pred_test(t_in, pred_func, feat):
    args = inspect.getfullargspec(pred_func).args
    if len(args)==1:
        return pred_func(t_in)
    elif len(args)==2:
        t_pred = pred_func(t_in,feat[args[1]]) 
    elif len(args)==3:
        t_pred = pred_func(t_in,feat[args[1]],feat[args[2]])
    return t_pred

def get_tile(img ,mask):
    try:
        m,n = img.shape
        a = img.copy().astype('int8')
        a[mask] = -1
        r=c=0
        for x in range(n):
            if np.count_nonzero(a[0:m,x]<0):continue
            for r in range(2,m):
                if 2*r<m and (a[0:r,x]==a[r:2*r,x]).all():break
            if r<m:break
            else: r=0
        for y in range(m):
            if np.count_nonzero(a[y,0:n]<0):continue
            for c in range(2,n):
                if 2*c<n and (a[y,0:c]==a[y,c:2*c]).all():break
            if c<n:break
            else: c=0
        if c>0:
            for x in range(n-c):
                if np.count_nonzero(a[:,x]<0)==0:
                    a[:,x+c]=a[:,x]
                elif np.count_nonzero(a[:,x+c]<0)==0:
                    a[:,x]=a[:,x+c]
        if r>0:
            for y in range(m-r):
                if np.count_nonzero(a[y,:]<0)==0:
                    a[y+r,:]=a[y,:]
                elif np.count_nonzero(a[y+r,:]<0)==0:
                    a[y,:]=a[y+r,:]
        return a[r:2*r,c:2*c]
    except:
        return a[0:1,0:1]
    
def patch_image(t_in,s_out,cm=0):
    try:
        t = t_in.copy()
        ty,tx=t.shape
        if cm>0:
            m = mask_rect(t==cm)
        else:
            m = (t==cm)   
        tile = get_tile(t ,m)
        if tile.size>2 and s_out==t.shape:
            rt = np.tile(tile,(1+ty//tile.shape[0],1+tx//tile.shape[1]))[0:ty,0:tx]
            if (rt[~m]==t[~m]).all():
                return rt
        for i in range(6):
            m = (t==cm)
            t -= cm
            if tx==ty:
                a = np.maximum(t,t.T)
                if (a[~m]==t[~m]).all():t=a.copy()
                a = np.maximum(t,np.flip(t).T)
                if (a[~m]==t[~m]).all():t=a.copy()
            a = np.maximum(t,np.flipud(t))
            if (a[~m]==t[~m]).all():t=a.copy()
            a = np.maximum(t,np.fliplr(t))
            if (a[~m]==t[~m]).all():t=a.copy()
            t += cm
            m = (t==cm)
            lms = measure.label(m.astype('uint8'))
            for l in range(1,lms.max()+1):
                lm = np.argwhere(lms==l)
                lm = np.argwhere(lms==l)
                x_min = max(0,lm[:,1].min()-1)
                x_max = min(lm[:,1].max()+2,t.shape[0])
                y_min = max(0,lm[:,0].min()-1)
                y_max = min(lm[:,0].max()+2,t.shape[1])
                gap = t[y_min:y_max,x_min:x_max]
                sy,sx=gap.shape
                if i==1:
                    sy//=2
                    y_max=y_min+sx
                gap = t[y_min:y_max,x_min:x_max]
                sy,sx=gap.shape
                allst = as_strided(t, shape=(ty,tx,sy,sx),strides=2*t.strides)    
                allst = allst.reshape(-1,sy,sx)
                allst = np.array([a for a in allst if np.count_nonzero(a==cm)==0])
                gm = (gap!=cm)
                for a in allst:
                    if sx==sy:
                        fpd = a.T
                        fad = np.flip(a).T
                        if i==1:gm[sy-1,0]=gm[0,sx-1]=False
                        if (fpd[gm]==gap[gm]).all():
                            gm = (gap!=cm)
                            np.putmask(gap,~gm,fpd)
                            t[y_min:y_max,x_min:x_max] = gap
                            break
                        if i==1:gm[0,0]=gm[sy-1,sx-1]=False
                        if (fad[gm]==gap[gm]).all():
                            gm = (gap!=cm)
                            np.putmask(gap,~gm,fad)
                            t[y_min:y_max,x_min:x_max] = gap
                            break 
                    fud = np.flipud(a)
                    flr = np.fliplr(a)
                    if i==1:gm[sy-1,0]=gm[0,sx-1]=gm[0,0]=gm[sy-1,sx-1]=False
                    if (a[gm]==gap[gm]).all():
                        gm = (gap!=cm)
                        np.putmask(gap,~gm,a)
                        t[y_min:y_max,x_min:x_max] = gap
                        break
                    elif (fud[gm]==gap[gm]).all():
                        gm = (gap!=cm)
                        np.putmask(gap,~gm,fud)
                        t[y_min:y_max,x_min:x_max] = gap
                        break
                    elif (flr[gm]==gap[gm]).all():
                        gm = (gap!=cm)
                        np.putmask(gap,~gm,flr)
                        t[y_min:y_max,x_min:x_max] = gap
                        break
        if s_out==t.shape:
            return t
        else:
            m = (t_in==cm)
            return np.resize(t[m],crop_min(m).shape)
    except:
        return []

## DSL Implementation

We start with the functions that take *one image* and produce an *a list of images*.](http://)

In [6]:
# np.array -> [np.array]
def groupByColor_unlifted(pixmap):
    """ Split an image into a collection of images with unique color """
    # Count the number of colors
    nb_colors = int(pixmap.max()) + 1
    # Create a pixmap for each color
    splited = [(pixmap == i) * i for i in range(1, nb_colors)]
    # Filter out empty images
    return [x for x in splited if np.any(x)]

# np.array -> [np.array]
def cropToContent_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    # Op argwhere will give us the coordinates of every non-zero point
    true_points = np.argwhere(pixmap)
    if len(true_points) == 0:
        return []
    # Take the smallest points and use them as the top left of our crop
    top_left = true_points.min(axis=0)
    # Take the largest points and use them as the bottom right of our crop
    bottom_right = true_points.max(axis=0)
    # Crop inside the defined rectangle
    pixmap = pixmap[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    return [pixmap]

# np.array -> [np.array]
def splitH_unlifted(pixmap):
    """ Split horizontally an image """
    h = pixmap.shape[0]
    if h % 2 == 1:
        h = h // 2
        return [pixmap[:h,:], pixmap[h+1:,:]]
    else:
        h = h // 2
        return [pixmap[:h,:], pixmap[h:,:]]

# np.array -> [np.array]
def negative_unlifted(pixmap):
    """ Compute the negative of an image (and conserve the color) """
    negative = np.logical_not(pixmap).astype(int)
    color = max(pixmap.max(), 1)
    return [negative * color]

# np.array -> [np.array]
def rotation1_unlifted(pixmap):
    """ Compute an image with rotation"""
        #90度反時計回り
    ans1 = np.rot90(np.array(pixmap))
    return [ans1]

# np.array -> [np.array]
def rotation2_unlifted(pixmap):
    """ Compute an image with rotation"""
    #180度反時計回り
    ans2 = np.rot90(np.array(pixmap), 2) 
    return [ans2]

# np.array -> [np.array]
def rotation3_unlifted(pixmap):
    """ Compute an image with rotation"""
    #270度反時計回り
    ans3 = np.rot90(np.array(pixmap), 3)
    return [ans3]
        
def unrotation1_unlifted(pixmap):
    #90度時計回り
    ans4 = np.rot90(np.array(pixmap),-1)
    return [ans4]

def unrotation2_unlifted(pixmap):
    #180度時計回り
    ans5 = np.rot90(np.array(pixmap), -2)
    return [ans5]

def unrotation3_unlifted(pixmap):
    #270度時計回り
    ans6 = np.rot90(np.array(pixmap), -3) 
    return [ans6]

# np.array -> [np.array]
def fliplr_unlifted(pixmap):
    """ Compute an image with flip"""
    #左右反転
    ans1 = np.fliplr(np.array(pixmap))
    return [ans1] 

# np.array -> [np.array]
def flipud_unlifted(pixmap):
    """ Compute an image with flip"""
    #上下反転
    ans2 = np.flipud(np.array(pixmap))
    return [ans2] 

# np.array -> [np.array]
def fliptr_unlifted(pixmap):
    """ Compute an image with flip"""
    #転置
    ans3 = np.transpose(np.array(pixmap))
    return [ans3]

# np.array -> [np.array]
def splitV_unlifted(pixmap):
    """ Split horizontally an image """
    v = pixmap.shape[1]
    if v % 2 == 1:
        v = v // 2
        return [pixmap[:,:v], pixmap[:,v+1:]]
    else:
        v = v // 2
        return [pixmap[:,:v], pixmap[:,v:]]

def copy_horizontal_unlifted(pixmap):
    """ copy picture and add horizontally """
    try: 
        return [pixmap.repeat(2, axis=0)]
    except:
        return []
    
def copy_vertical_unlifted(pixmap):
    """ copy picture and add vertically """
    try:
        return [pixmap.repeat(2, axis=1)]
    except:
        return []

def crop_inside_unlifted(pixmap):
    """ Crop inside an image to fit exactly the non 0 pixels """
    # Op argwhere will give us the coordinates of every non-zero point
    try:
        true_points = np.argwhere(pixmap)
        if len(true_points) == 0:
            return []
        # Take the smallest points and use them as the top left of our crop
        top_left = true_points.min(axis=0)
        # Take the largest points and use them as the bottom right of our crop
        bottom_right = true_points.max(axis=0)
        # Crop inside the defined rectangle
        pixmap = pixmap[top_left[0]+1:bottom_right[0], top_left[1]+1:bottom_right[1]]
        return [pixmap]
    except:
        return []

def expand_twice_unlifted(arr):
    try:
        return [arr.repeat(2, axis=1).repeat(2,axis=0)]
    except:
        return []

def expand_three_unlifted(arr):
    try:
        return [arr.repeat(3, axis=1).repeat(3,axis=0)]
    except:
        return []
    
def expand_four_unlifted(arr):
    try:
        return [arr.repeat(4, axis=1).repeat(4,axis=0)]
    except:
        return []

def horizontal_combi_twice_unlifted(arr):
    if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
        org = arr[0]
        return [org]
    elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
        return [np.zeros([3,3])]
    elif str(type(arr)) == "<class 'numpy.ndarray'>":
        ans = np.hstack([arr,arr])
        return [ans]
    else:
        return [np.zeros([3,3])]

def horizontal_combi_three_unlifted(arr):
    if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
        org = arr[0]
        return [org]
    elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
        return [np.zeros([3,3])]
    elif str(type(arr)) == "<class 'numpy.ndarray'>":
        ans = np.hstack([arr,arr,arr])
        return [ans]
    else:
        return [np.zeros([3,3])]

def vertical_combi_twice_unlifted(arr):
    try:
        return [np.concatenate((arr,arr),axis=1)]
    except:
        return []

def vertical_combi_three_unlifted(arr):
    try:
        return [np.concatenate((arr,arr,arr),axis=1)]
    except:
        return []

def kronecker_expansion_unlifted(arr):
    try:
        flg = arr > 0
        return [np.kron(flg,arr)]
    except:
        return []

def expansion_fliplr_unlifted(pixmap):
    """ Expand an image by connecting flipped image"""
    try: 
        tmp = np.fliplr(np.array(pixmap))
        ans1 = np.concatenate((pixmap, tmp), axis=1)
        ans2 = np.concatenate((tmp, pixmap), axis=1)
        return [ans1, ans2]
    except:
        return []

def expansion_flipud_unlifted(pixmap):
    """ Expand an image by connecting flipped image"""
    try:
        tmp = np.flipud(np.array(pixmap))
        ans1 = np.concatenate((pixmap, tmp), axis=0)
        ans2 = np.concatenate((tmp, pixmap), axis=0)
        return [ans1, ans2]
    except:
        return []

# np.array -> [np.array]
def overlapV_unlifted(pixmap):
    """ Split vertically an image and overlap"""
    try:
        v = pixmap.shape[1]
        if v % 2 == 1:
            v = v // 2
            a, b = pixmap[:,:v], pixmap[:,v+1:]
        else:
            v = v // 2
            a, b = pixmap[:,:v], pixmap[:,v:]
        flg_a = a > 0
        flg_b = b > 0
        return [np.logical_and(flg_a, flg_b).astype("int32"), np.logical_or(flg_a, flg_b).astype("int32"), np.logical_xor(flg_a, flg_b).astype("int32")]
    except:
        return []

# np.array -> [np.array]
def overlapH_unlifted(pixmap):
    """ Split horizontally an image and overlap"""
    try: 
        h = pixmap.shape[0]
        if h % 2 == 1:
            h = h // 2
            a, b = [pixmap[:h,:], pixmap[h+1:,:]]
        else:
            h = h // 2
            a, b = [pixmap[:h,:], pixmap[h:,:]]
        flg_a = a > 0
        flg_b = b > 0
        return [np.logical_and(flg_a, flg_b).astype("int32"), np.logical_or(flg_a, flg_b).astype("int32"), np.logical_xor(flg_a, flg_b).astype("int32")]
    except:
        return []

def three_diagonal_pattern_unlifted(x):
    try:
        H, W = x.shape
        colors = [0, 0, 0]
        for yy in range(H):
            for xx in range(W):
                color = x[yy, xx]
                if color != 0:
                    colors[(yy+xx)%3] = color
        y = x.copy()
        for yy in range(H):
            for xx in range(W):
                y[yy, xx] = colors[(yy+xx)%3]
        return [y]
    except:
        return []

def expand_by_unique_colors_unlifted(pixmap):
    """ copy picture and add horizontally """
    try: 
        x = pixmap.copy()
        unique_num = len(np.unique(x[x > 0]))
        return [x.repeat(unique_num, axis=0).repeat(unique_num, axis=1)]
    except:
        return []

def kronecker_negative_expansion_unlifted(arr):
    try:
        flg = arr > 0
        negative = np.logical_not(arr).astype(int)
        color = max(arr.max(), 1)
        tmp = negative * color
        return [np.kron(flg,tmp)]
    except:
        return []

def kronecker_nn_expansion_unlifted(arr):
    try:
        negative = np.logical_not(arr).astype(int)
        color = max(arr.max(), 1)
        tmp = negative * color
        flg = tmp > 0
        return [np.kron(flg,tmp)]
    except:
        return []

def kronecker_minimum_position_unlifted(arr):
    try:
        unique, counts = np.unique(arr, return_counts=True)
        num_dict = dict(zip(unique, counts))
        mini = min(num_dict.values())
        for i in num_dict.keys():
            if num_dict[i] == mini:
                num_element = i
        flg = arr == num_element
        return [np.kron(flg, arr)]
    except:
        return []

def kronecker_maximum_position_unlifted(arr):
    try:
        unique, counts = np.unique(arr, return_counts=True)
        num_dict = dict(zip(unique, counts))
        maxi = max(num_dict.values())
        for i in num_dict.keys():
            if num_dict[i] == maxi:
                num_element = i
        flg = arr == num_element
        return [np.kron(flg, arr)]
    except:
        return []

def crop_from_left_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    try:
        ans = []
        num_list = []
        H, W = pixmap.shape
        for i in range(W):
            unique, counts = np.unique(pixmap[:,i], return_counts=True)
            num_dict = dict(zip(unique, counts))
            num_dict.pop(0, None)
            for j in list(num_dict.keys()):
                if j not in num_list:
                    num_list.append(j)
        for i in num_list:
           # Op argwhere will give us the coordinates of every non-zero point
            true_points = np.argwhere(pixmap==i)
            if len(true_points) == 0:
                continue
           # Take the smallest points and use them as the top left of our crop
            top_left = true_points.min(axis=0)
           # Take the largest points and use them as the bottom right of our crop
            bottom_right = true_points.max(axis=0)
           # Crop inside the defined rectangle
            tmp = pixmap[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
            ans.append(tmp)
        return ans
    except:
        return []

def crop_from_top_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    try: 
        ans = []
        num_list = []
        H, W = pixmap.shape
        for i in range(H):
            unique, counts = np.unique(pixmap[i,:], return_counts=True)
            num_dict = dict(zip(unique, counts))
            num_dict.pop(0, None)
            for j in list(num_dict.keys()):
                if j not in num_list:
                    num_list.append(j)
        for i in num_list:
           # Op argwhere will give us the coordinates of every non-zero point
            true_points = np.argwhere(pixmap==i)
            if len(true_points) == 0:
                continue
           # Take the smallest points and use them as the top left of our crop
            top_left = true_points.min(axis=0)
           # Take the largest points and use them as the bottom right of our crop
            bottom_right = true_points.max(axis=0)
           # Crop inside the defined rectangle
            tmp = pixmap[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
            ans.append(tmp)
        return ans
    except:
        return []

def split_quarter_unlifted(pixmap):
    """ Split quarterlly an image """
    try: 
        h, v = pixmap.shape
        if h%2==1 and v%2 ==1:
            h = h//2
            v = v//2
            return [pixmap[:h,:v], pixmap[:h,v+1:], pixmap[h+1:,:v], pixmap[h+1:,v+1:]]
        elif h%2==0 and v%2 ==1:
            h = h//2
            v = v//2
            return [pixmap[:h,:v], pixmap[:h,v+1:], pixmap[h:,:v], pixmap[h:,v+1:]]
        elif h%2==1 and v%2==0:
            h = h//2
            v = v//2
            return [pixmap[:h,:v], pixmap[:h,v:], pixmap[h+1:,:v], pixmap[h+1:,v:]]
        else:
            h = h//2
            v = v//2
            return [pixmap[:h,:v], pixmap[:h,v:], pixmap[h:,:v], pixmap[h:,v:]]
    except:
        return []
    
def expansion_negative_fliplr_unlifted(pixmap):
    try: 
        w = pixmap.shape[1]
        tmp = np.fliplr(np.array(pixmap))
        negative = np.logical_not(tmp).astype(int)
        color = max(tmp.max(), 1)
        tmp = negative * color
        return [np.concatenate([tmp, pixmap], axis=1), np.concatenate([pixmap, tmp], axis=1)]
    except:
        return []

def expansion_negative_flipud_unlifted(pixmap):
    try:
        w = pixmap.shape[1]
        tmp = np.flipud(np.array(pixmap))
        negative = np.logical_not(tmp).astype(int)
        color = max(tmp.max(), 1)
        tmp = negative * color
        return [np.concatenate([tmp, pixmap], axis=0), np.concatenate([pixmap, tmp], axis=0)]
    except:
        return []

def expansion_by_unique_num_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    try: 
        ans = pixmap.copy()
        unique, counts = np.unique(pixmap, return_counts=True)
        num_dict = dict(zip(unique, counts))
        num_dict.pop(0, None)
        num = len(num_dict)
        for i in range(num-1):
            ans = np.concatenate([ans, pixmap], axis=1)
        tmp = ans.copy()
        for i in range(num-1):
            ans = np.concatenate([ans, tmp], axis=0)
        return [ans]
    except:
        return []

def clockwise_expansion_unlifted(pixmap):
    h,w = pixmap.shape
    try:
        upper_right = np.rot90(np.array(pixmap))
        lower_right = np.rot90(np.array(pixmap),-1)
        lower_left = np.rot90(np.array(pixmap), -2)
        upper = np.concatenate([pixmap, upper_right], axis=1)
        lower = np.concatenate([lower_left, lower_right], axis=1)
        return [np.concatenate([upper, lower], axis=0)]
    except:
        return []

def Hoverlap_check_unlifted(pixmap):
    try: 
        v = pixmap.shape[1]
        if v % 2 == 1:
            v = v // 2
            a, b = pixmap[:,:v], pixmap[:,v+1:]
        else:
            v = v // 2
            a, b = pixmap[:,:v], pixmap[:,v:]
        flg_a = a > 0
        flg_b = b > 0
        if np.any(np.logical_and(flg_a, flg_b).astype("int32")):
            return [a]
        else:
            return [a+b]
    except:
        return []
    
def Voverlap_check_unlifted(pixmap):
    try: 
        h = pixmap.shape[0]
        if h % 2 == 1:
            h = h // 2
            a, b = [pixmap[:h,:], pixmap[h+1:,:]]
        else:
            h = h // 2
            a, b = [pixmap[:h,:], pixmap[h:,:]]
        flg_a = a > 0
        flg_b = b > 0
        if np.any(np.logical_and(flg_a, flg_b).astype("int32")):
            return [a]
        else:
            return [a+b]
    except:
        return []
    
def horizontal_combi_five_unlifted(arr):
    try:
        tmp = arr.copy()
        for i in range(2):
            tmp = np.concatenate([tmp,np.fliplr(tmp)], axis=1)
        ans = np.concatenate([tmp, arr], axis=1)
        return [ans]
    except:
        return []

def expansion_8directions_unlifted(pixmap):
    try:
        tmp_ud = np.flipud(np.array(pixmap))
        tmp_lr = np.fliplr(np.array(pixmap))
        tmp_corner = np.rot90(np.array(pixmap),2)
        top = np.concatenate([tmp_corner, tmp_ud, tmp_corner], axis=1)
        middle = np.concatenate([tmp_lr, pixmap, tmp_lr], axis=1)
        return [np.concatenate([top, middle, top], axis=0)]
    except:
        return []

def addition_by_size_unlifted(pixmap):
    """ copy picture and add horizontally """
    try:
        H = pixmap.shape[0]
        tmp = pixmap.copy()
        for _ in range(H-1):
            tmp = np.concatenate([tmp, pixmap], axis=1)
        ans = tmp.copy()
        for _ in range(H-1):
            ans = np.concatenate([ans, tmp], axis=0)
        return [ans]
    except:
        return []

def pickup_objects_unlifted(pixmap):
    try:
        loc = np.argwhere(pixmap > 0)
        h, w = pixmap.shape
        count = 0
        groups = [] # pick up coordinates
        ans = [] # pick up objects
        if len(loc) > 0:
            while True:
                if count == 0:
                    tmp, remain = make_group(loc, h, w)
                else:
                    tmp, remain = make_group(remain, h, w)
                groups.append(tmp)
                count += 1
                if len(remain) == 0:
                    break
            for i in groups:    
                x_min = np.min(np.array(i)[:,0]) # x coordinate np.array(i)[:,0]
                x_max = np.max(np.array(i)[:,0]) # x coordinate
                y_min = np.min(np.array(i)[:,1]) # y coordinate np.array(i)[:,1]
                y_max = np.max(np.array(i)[:,1]) # y coordinate
                tmp = pixmap[x_min:x_max+1, y_min:y_max+1]
                ans.append(tmp)
            return ans
        else:
            return []
    except:
        return []
    
def make_copy_in_right_side_unlifted(pixmap):
    try:
        points = np.argwhere(pixmap>0).tolist()
        color_num = np.max(pixmap)
        max_x = np.max(np.array(points)[:,1])
        tmp = points.copy()
        for i in tmp:
            i_after = 2 * (max_x -i[1]) + 1 + i[1]
            if i_after < pixmap.shape[1]:
                pixmap[i[0],i_after]=color_num
        return [pixmap]
    except:
        return []
    
def find_separation_lines_unlifted(pixmap):
    try:
        h,w = pixmap.shape
        horizontal_loc = []
        for i in range(h):
            if len(np.unique(pixmap[i,:])) == 1:
                horizontal_loc.append(i)
        vertical_loc = []
        for j in range(w):
            if len(np.unique(pixmap[:,j])) == 1:
                vertical_loc.append(j)
        ans = np.zeros([len(horizontal_loc)+1, len(vertical_loc)+1])
        h_element = [i for i in range(h) if i not in horizontal_loc]
        w_element = [i for i in range(w) if i not in vertical_loc]
        colors = [pixmap[i][j] for i in h_element for j in w_element]
        
        if len(np.unique(colors)) == 1:
            for i in range(len(horizontal_loc)+1):
                for j in range(len(vertical_loc)+1):
                    ans[i][j] = colors[0]
            return [ans]
        else:
            for i in range(len(horizontal_loc)):
                for j in range(len(vertical_loc)): 
                    if i == len(horizontal_loc)-1:
                        if j == len(vertical_loc)-1:
                            ans[i,j] = np.unique(pixmap[horizontal_loc[i-1]+1:horizontal_loc[i], vertical_loc[j-1]+1:vertical_loc[j]])[0] 
                            ans[i+1,j] = np.unique(pixmap[horizontal_loc[i-1]+1:horizontal_loc[i], vertical_loc[j]+1:])[0] 
                            ans[i, j+1] = np.unique(pixmap[horizontal_loc[i]+1:, vertical_loc[j-1]+1:vertical_loc[j]])[0] 
                            ans[i+1, j+1] = np.unique(pixmap[horizontal_loc[i]+1:, vertical_loc[j]+1:])[0] 
                    elif j == 0:
                        ans[i,j] = np.unique(pixmap[horizontal_loc[i-1]+1:horizontal_loc[i], :vertical_loc[j]])[0] 
                        ans[i+1,j] = np.unique(pixmap[horizontal_loc[i]+1:, :vertical_loc[j]])[0] 
                    else:
                        ans[i,j] = np.unique(pixmap[horizontal_loc[i-1]+1:horizontal_loc[i], vertical_loc[j-1]+1:vertical_loc[j]])[0] 
                        ans[i+1,j] = np.unique(pixmap[horizontal_loc[i]+1:, vertical_loc[j-1]+1:vertical_loc[j]])[0]          
                else:
                    if j == 0:
                        ans[i,j] = np.unique(pixmap[:horizontal_loc[i], :vertical_loc[j]])[0]
                    elif j == len(vertical_loc)-1:
                        ans[i,j] = np.unique(pixmap[:horizontal_loc[i], vertical_loc[j-1]+1:vertical_loc[j]])[0] 
                        ans[i,j+1] = np.unique(pixmap[:horizontal_loc[i], vertical_loc[j]+1:])[0] 
                    else:
                        ans[i,j] = np.unique(pixmap[:horizontal_loc[i], vertical_loc[j-1]+1:vertical_loc[j]])[0]            
        return [ans]
    except:
        return []
    
def unique_center_to_edge_unlifted(pixmap):
    try:
        ans = np.zeros_like(pixmap)
        unique, counts = np.unique(pixmap, return_counts=True)
        num_dict = dict(zip(unique, counts))
        num_dict.pop(0, None)
        if len(num_dict) > 1:
            return []
        else:
            h,w = ans.shape
            ans[h-1,:] = list(num_dict.values())[0]
            ans[0,:] = list(num_dict.values())[0]
            ans[:,w-1] = list(num_dict.values())[0]
            ans[:,0] = list(num_dict.values())[0]
            return [ans]
    except:
        return []
    
def closed_area_to_yellow_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["yellow"]
    return [y]

def closed_area_to_blue_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["blue"]
    return [y]

def closed_area_to_red_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["red"]
    return [y]

def closed_area_to_green_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["green"]
    return [y]

def closed_area_to_orange_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["orange"]
    return [y]

def gray_hanten_unlifted(pixmap):
    try:
        ans = pixmap.copy()
        uniques = np.unique(np.array(pixmap)).tolist()
        another_color = [i for i in uniques if i != 5][0]
        gray_loc = np.array(np.where(pixmap == color2num["gray"])).T.tolist()
        other_loc = np.array(np.where(pixmap == another_color)).T.tolist()
        for loc in gray_loc:
            ans[loc[0],loc[1]] = another_color
        for loc in other_loc:
            ans[loc[0],loc[1]] = 0
        return [ans]
    except:
        return []
    
def gray_copy_unlifted(pixmap):
    try:
        ans = pixmap.copy()
        all_areas = pickup_objects_unlifted(pixmap)
        gray_coor, gray_areas = pickup_gray_area(pixmap)
        original = [np.array(i) for i in all_areas if np.sum(np.array(i)!=5) > 0][0]
        for area in gray_coor:
            ans[area[0]:area[1], area[2]:area[3]] = np.array(original)
        return [ans]
    except:
        return []
    
def change_gray_area_unlifted(pixmap):
    try:
        ans = pixmap.copy()
        gray_coor, gray_areas = pickup_gray_area(pixmap)
        length = [np.array(i).shape[0]*np.array(i).shape[1] for i in gray_areas]
        for coor, area in zip(gray_coor, gray_areas):
            if np.array(area).shape[0] * np.array(area).shape[1] == min(length):
                for i in range(coor[0],coor[1]):
                    for j in range(coor[2], coor[3]):
                        ans[i, j] = color2num["red"]
            elif np.array(area).shape[0] * np.array(area).shape[1] == max(length):
                for i in range(coor[0],coor[1]):
                    for j in range(coor[2], coor[3]):
                        ans[i, j] = color2num["blue"]
            else:
                for i in range(coor[0],coor[1]):
                    for j in range(coor[2], coor[3]):
                        ans[i, j] = color2num["yellow"]
        return [ans]
    except:
        return []
    
def vertical_combi_three_another_unifted(arr):
    if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
        org = arr[0]
        a = np.flipud(org)
        return [np.vstack([org,a,org])]
    elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
        return [np.zeros([3,3])]
    elif str(type(arr)) == "<class 'numpy.ndarray'>":
        a = np.flipud(arr)
        return [np.vstack([arr,a,arr])]
    else:
        return [np.zeros([3,3])]
    
def new_formation_unifted(arr):
    try:
        if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
            org = arr[0]
            a = np.fliplr(org)
            part1 = np.hstack([org,org,org])
            part2 = np.hstack([a,a,a])
            return [np.vstack([part1,part2,part1])]
        elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
            return [np.zeros([3,3])]
        elif str(type(arr)) == "<class 'numpy.ndarray'>":
            a = np.fliplr(arr)
            part1 = np.hstack([arr,arr,arr])
            part2 = np.hstack([a,a,a])
            return [np.vstack([part1,part2,part1])]
        else:
            return [np.zeros([3,3])]
    except:
        []
        
def new_formation2_unlifted(pixmap):
    try:
        part1 = np.flipud(pixmap)
        right = np.vstack([part1, pixmap])
        left = np.fliplr(right)
        return [np.hstack([left,right])]
    except:
        return []
    
def kronecker_straight_unlifted(arr):
    try:
        h, w = arr.shape
        flg = 0
        for row in range(h):
            if len(np.unique(arr[row,:]))==1:
                color = arr[row,0]
                flg = 1
                break
        if flg == 0:
            for col in range(w):
                if len(np.unique(arr[:,col]))==1:
                    color = arr[0,col]
                    flg = 1
                    break
                
        tmp = arr == color
        return [np.kron(tmp, arr)]
    except:
        return []

def blue_red_unlifted(arr):
    try: 
        if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
            org = arr[0]
            h,w = org.shape
            for i in range(0,h,2):
                for j in range(0,w,2):
                    if org[i][j] >0:
                        org[i][j] = color2num["blue"]
                        org[i+1][j] = color2num["red"]
                        org[i][j] = color2num["blue"]
                        org[i+1][j] = color2num["red"]
            return [org]
        elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
            return [np.zeros([3,3])]
        elif str(type(arr)) == "<class 'numpy.ndarray'>":
            h,w = arr.shape
            for i in range(0,h,2):
                for j in range(0,w,2):
                    if arr[i][j] >0:
                        arr[i][j] = color2num["blue"]
                        arr[i][j+1] = color2num["red"]
                        arr[i+1][j+1] = color2num["blue"]
                        arr[i+1][j] = color2num["red"]
            return [arr]
        else:
            return [np.zeros([3,3])]
    except:
        return []
    
def overlap_color_objects_unlifted(arr):
    try:
        ans = []
        size = []
        colorlist = np.unique(np.array(arr)).tolist()
        for color in colorlist:
            res = arr == color
            res = np.array(pickup_objects_unlifted(res)[0])
            h, w = res.shape
            tmp = np.zeros([h,w])
            for i in range(h):
                for j in range(w):
                    tmp[i][j] = color
            ans.append(tmp)
            size.append(h*w)
        cons_index = sorted(range(len(size)), key=lambda k: -size[k])
        for i in range(len(cons_index)):
            if i == 0:
                pass
            elif i == 1:
                org = ans[cons_index[i]]
            else:
                tmp = ans[cons_index[i]]
                h,w = tmp.shape
                org[:h,:w] = tmp
        return [org]
    except:
        return []
    
def new_expansion_unlifted(arr):
    try:
        if str(type(arr)) == "<class 'list'>" and len(arr) == 1:
            org = arr[0]
            h, w = np.array(org).shape
            ans = np.zeros([2*w + h, 2*w + h])
            ans[w:w+h,0:w] = org
            ans[w:w+h,w+h:2*w+h] = np.fliplr(org)
            ans[0:w,w:w+h] = np.rot90(org,-1)
            ans[w+h:2*w+h,w:w+h] = np.rot90(org)
            return [ans]
        elif str(type(arr)) == "<class 'list'>" and len(arr) >= 2:
            return [np.zeros([3,3])]
        elif str(type(arr)) == "<class 'numpy.ndarray'>":
            h,w = arr.shape
            ans = np.zeros([2*w + h, 2*w + h])
            ans[w:w+h,0:w] = arr
            ans[w:w+h,w+h:2*w+h] = np.fliplr(arr)
            ans[0:w,w:w+h] = np.rot90(arr,-1)
            ans[w+h:2*w+h,w:w+h] = np.rot90(arr)
            return [ans]
        else:
            return [np.zeros([3,3])]
    except:
        return []

We now write functions that take a list of images and transform it to a new list. ([np.array] -> [np.array])

In [7]:
def identity(x: [np.array]):
    return x

def tail(x):
    if len(x) > 1:
        return x[1:]
    else:
        return x

def init(x):
    if len(x) > 1:
        return x[:1]
    else:
        return x

def union(x):
    """ Compute the pixel union of all images in the list. """
    try: 
        if len(x) < 2:
            return x
    
        # Make sure everybody have the same shape
        first_shape = tuple(x[0].shape)
        for pixmap in x[1:]:
            if first_shape != tuple(pixmap.shape):
                return []
    
        return [np.bitwise_or.reduce(np.array(x).astype(int))]
    except:
        return []

def intersect(x):
    """ Compute the pixel intersection of all images in the list. """
    if len(x) < 2:
        return x
    
    # Make sure everybody have the same shape
    first_shape = tuple(x[0].shape)
    for pixmap in x[1:]:
        if first_shape != tuple(pixmap.shape):
            return []
    
    return [(np.prod(np.array(x), axis=0) > 0).astype(int)]

def sortByColor(xs):
    """ Sort pictures by increasing color id. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: x.max()))

def sortByWeight(xs):
    """ Sort images by how many non zero pixels are contained. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: (x>0).sum()))

def reverse(x):
    """ Reverse the order of a list of images. """
    return x[::-1]

def connect_inlist(pixmap):
    try:
        ans =[]
        if len(pixmap) >=2:
            row_len = [pixmap[i].shape[0] for i in range(len(pixmap))]
            col_len = [pixmap[i].shape[1] for i in range(len(pixmap))]
            if len(np.unique(row_len))==1:
                ans_horizon = pixmap[0]
                for i in range(1, len(pixmap)):
                    ans_horizon = np.concatenate([ans_horizon, pixmap[i]], axis=1)
                ans.append(ans_horizon)
            if len(np.unique(col_len))==1:
                ans_vertical = pixmap[0]
                for i in range(1, len(pixmap)):
                    ans_vertical = np.concatenate([ans_vertical, pixmap[i]], axis=0)
                ans.append(ans_vertical)
        return ans
    except:
        return []
    
def find_horizontal_asymmetric_objects(x):
    try: 
        ans = []
        for picture in x:
            h, w = picture.shape
            w = w//2
            for i in range(w):
                if np.all(picture[:,i] == picture[:,w-1-i]):
                    continue
                else:
                    ans.append(picture)
                    break
        return ans
    except:
        return []

def find_horizontal_symmetric_objects(x):
    try:
        ans = []
        for picture in x:
            flg = 0
            h, w = picture.shape
            w = w//2
            for i in range(w):
                if np.all(picture[:,i] == picture[:,w-1-i]) != 1:
                    flg = 1
                    break
            if flg == 0:
                ans.append(picture)       
        return ans
    except:
        return []

def find_vertical_asymmetric_objects(x):
    try:
        ans = []
        for picture in x:
            h, w = picture.shape
            h = h//2
            for i in range(h):
                if np.all(picture[i,:] == picture[h-1-i,:]):
                    continue
                else:
                    ans.append(picture)
                    break
        return ans
    except:
        return []

def find_vertical_symmetric_objects(x):
    try:
        ans = []
        for picture in x:
            flg = 0
            h, w = picture.shape
            h = h//2
            for i in range(h):
                if np.all(picture[i,:] == picture[h-1-i,:]) != 1:
                    flg = 1
                    break
            if flg == 0:
                ans.append(picture)       
        return ans
    except:
        return []

def find_element_with_smallest_noise_objects(x):
    try:
        ans = []
        nums = []
        for picture in x:
            unique, counts = np.unique(picture, return_counts=True)
            num_dict = dict(zip(unique, counts))
            nums.append(sum(num_dict.values()) - max(num_dict.values()))
        mini_ind = [i for i in range(len(nums)) if nums[i] == min(nums)]
        for i in mini_ind:
            ans.append(x[i])
        return ans
    except:
        return []
    
def find_element_with_largest_noise_objects(x):
    try:
        ans = []
        nums = []
        for picture in x:
            unique, counts = np.unique(picture, return_counts=True)
            num_dict = dict(zip(unique, counts))
            nums.append(sum(num_dict.values()) - max(num_dict.values()))
        maxi_ind = [i for i in range(len(nums)) if nums[i] == max(nums)]
        for i in maxi_ind:
            ans.append(x[i])
        return ans
    except:
        return []

def find_objects_with_largest_uniques(x):
    try:
        ans = []
        nums = []
        for picture in x:
            unique, counts = np.unique(picture, return_counts=True)
            num_dict = dict(zip(unique, counts))
            nums.append(len(num_dict))
        maxi_ind = [i for i in range(len(nums)) if nums[i] == max(nums)]
        for i in maxi_ind:
            ans.append(x[i])
        return ans
    except:
        return []

def find_objects_with_smallest_uniques(x):
    try:
        ans = []
        nums = []
        for picture in x:
            unique, counts = np.unique(picture, return_counts=True)
            num_dict = dict(zip(unique, counts))
            nums.append(len(num_dict))
        mini_ind = [i for i in range(len(nums)) if nums[i] == min(nums)]
        for i in mini_ind:
            ans.append(x[i])
        return ans
    except:
        return []

def find_unique_objects(pixmap):
    try:
        if len(pixmap) >=3:
            ans = []
            for tmp in pixmap:
                check = [np.all(tmp == j) for j in ans]
                if np.any(check):
                    true_position = [k for k in range(len(check)) if check[k]][0]
                    ans.pop(true_position)
                else:
                    ans.append(tmp)
            return ans
        else:
            return []
    except:
        return []
    
def find_same_objects(pixmap):
    try:
        if len(pixmap) >=3:
            uniques = find_unique_objects(pixmap)
            ans = pixmap.copy()
            for u in uniques:
                check = [np.all(tmp == u) for tmp in ans]
                true_position = [k for k in range(len(check)) if check[k]][0]
                ans.pop(true_position)
            if len(ans) >0:
                return [ans[0]]
            else:
                return []
        else:
            return []
    except:
        return []
    
def overlap_by_maximum_value(pixmap):
    def select_bigger_element_in_matrix(a,b):
        ans = a.copy()
        h, w = a.shape
        for i in range(h):
            for j in range(w):
                ans[i][j] = max(a[i][j], b[i][j])
        return ans
    try:
        col_nums = [i.shape[1] for i in pixmap]
        row_nums = [i.shape[0] for i in pixmap]
        if len(np.unique(col_nums)) == 1 and len(np.unique(row_nums)) == 1 and len(pixmap)>=2:
            ans = pixmap[0]
            for i in range(1,len(pixmap)):
                ans = select_bigger_element_in_matrix(ans, pixmap[i])
            return [ans]
        else:
            return []
    except:
        return []
    
def overlap_by_minimum_value(pixmap):
    def select_bigger_element_in_matrix(a,b):
        ans = a.copy()
        h, w = a.shape
        for i in range(h):
            for j in range(w):
                ans[i][j] = min(a[i][j], b[i][j])
        return ans
    col_nums = [i.shape[1] for i in pixmap]
    row_nums = [i.shape[0] for i in pixmap]
    if len(np.unique(col_nums)) == 1 and len(np.unique(row_nums)) == 1 and len(pixmap)>=2:
        ans = pixmap[0]
        for i in range(1,len(pixmap)):
            ans = select_bigger_element_in_matrix(ans, pixmap[i])
        return [ans]
    else:
        return []
    
def represent_only_inside(pixmap):
    try: 
        ans = []
        for pic in pixmap:
            points = sorted(np.argwhere(pic>0).tolist())
            inside = find_inside(points)
            color = np.max(pic)
            pic[:,:] = 0
            for i in inside:
                pic[i[0],i[1]] = color
            ans.append(pic)
        return ans
    except:
        return []
    
def combine_into_one_piece(pixmap):
    try:
        dim = [x.shape[0] * x.shape[1] for x in pixmap]
        sorted_ids = sorted(range(len(dim)), key=lambda k: dim[k], reverse=True)
        original_pic_id = sorted_ids[0]
        other_ids = [i for i in sorted_ids if i != original_pic_id]
        ans = pixmap[original_pic_id].copy()
        h, w = ans.shape
        for i in other_ids:
            pixmap[i][pixmap[i]==0]=np.max(ans)
        for i in other_ids:
            flg = False
            sample_h, sample_w = pixmap[i].shape
            for row in range(h):
                for col in range(w):
                    if row+sample_h>h or col+sample_w>w:
                        continue
                    else:
                        zero_num = np.sum(ans[row:row+sample_h,col:col+sample_w] == 0)
                        non_zero_after_combi = np.sum(pixmap[i]-ans[row:row+sample_h,col:col+sample_w]!=0)
                        if zero_num == non_zero_after_combi:
                            flg = True
                            ans[row:row+sample_h,col:col+sample_w] = pixmap[i]
                            break
                if flg:
                    break
        return [ans]
    except:
        return []
    
def combine_into_one_piece2(pixmap):
    try:
        dim = [x.shape[0] * x.shape[1] for x in pixmap]
        sorted_ids = sorted(range(len(dim)), key=lambda k: dim[k], reverse=True)
        original_pic_id = sorted_ids[0]
        other_ids = [i for i in sorted_ids if i != original_pic_id]
        ans = pixmap[original_pic_id].copy()
        unique, counts = np.unique(ans, return_counts=True)
        num_dict = dict(zip(unique, counts))
        change_num = max(num_dict, key=num_dict.get) #https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
        ans[ans==change_num] = 0
        h, w = ans.shape
        for i in other_ids:
            basic_color = np.max(pixmap[i]) 
            flg = False
            for row in range(h):
                for col in range(w):
                    if ans[row][col]==basic_color:
                        ans[row-1:row+2,col-1:col+2] = pixmap[i]
                        flg = True
                        break
                if flg:
                    break
        return [ans]
    except:
        return []
    
def combine_hole_objects(pixmap):
    # https://note.nkmk.me/python-math-factorial-permutations-combinations/
    try:
        tmp = []
        ans = []
        for i in pixmap:
            if np.min(i) == 0:
                 tmp.append(i)
        for v in itertools.permutations(range(len(tmp)), len(tmp)):
            for i in range(len(tmp)):
                if i == 0:
                    first_h = tmp[v[i]]
                    first_v = tmp[v[i]]
                else:
                    first_h = np.concatenate([first_h, tmp[v[i]]], axis=1)
                    first_v = np.concatenate([first_v, tmp[v[i]]], axis=0)
            ans.append(first_h)
            ans.append(first_v)
        return ans
    except:
        return []
    
def copy_and_chng_color(pixmap):
    size = [i.shape[0] *i.shape[1] for i in pixmap]
    max_ind = [i for i in range(len(size))  if size[i] == max(size)][0]
    pics = []
    ans = []
    for i in range(len(pixmap)):
        if i != max_ind:
            color = np.unique(pixmap[i])[0]
            tmp = pixmap[max_ind].copy()
            h,w = tmp.shape
            for row in range(h):
                for col in range(w):
                    if tmp[row][col] != 0:
                        tmp[row][col] = color
            pics.append(tmp)
    if len(np.unique([i.shape[0] for i in pics])) == 1:
        tmp = pics[0]
        for i in range(1,len(pics)):
            tmp = np.hstack([tmp,pics[i]])
        ans.append(tmp)
    if len(np.unique([i.shape[1] for i in pics])) == 1:
        tmp = pics[0]
        for i in range(1,len(pics)):
            tmp = np.vstack([tmp,pics[i]])
        ans.append(tmp)
    return ans

## Composition of functions

It is important to make sure we can chain both functions. To compose two functions `f` and `g` of type `[np.array] -> [np.array]` ; We symply call `g(f([input_image]))`.

But for each function of the type `np.array -> [np.array]` some work is required. We need to generated a *lifted version* version of them. A function `f: np.array -> [np.array]` can be turned into a function of type `[np.array] -> [np.array]` by applying `f` on each image of the input list and concatenating the results.

If you want to know more about the `lift` concept, have a look to the concept of [*monades*](https://en.wikipedia.org/wiki/Monad_%28functional_programming%29). We are indeed using the *list monade*.

In [8]:
def lift(fct):
    # Lift the function
    def lifted_function(xs):
        list_of_results = [fct(x) for x in xs]
        return list(itertools.chain(*list_of_results))
    # Give a nice name to the lifted function
    import re
    lifted_function.__name__ = re.sub('_unlifted$', '_lifted', fct.__name__)
    return lifted_function

cropToContent = lift(cropToContent_unlifted)
groupByColor = lift(groupByColor_unlifted)
splitH = lift(splitH_unlifted)
negative = lift(negative_unlifted)
rotation1 = lift(rotation1_unlifted)
rotation2 = lift(rotation2_unlifted)
rotation3 = lift(rotation3_unlifted)
unrotation1 = lift(unrotation1_unlifted)
unrotation2 = lift(unrotation2_unlifted)
unrotation3 = lift(unrotation3_unlifted)
fliplr = lift(fliplr_unlifted)
flipud = lift(flipud_unlifted)
fliptr = lift(fliptr_unlifted)
splitV = lift(splitV_unlifted)
copy_horizontal = lift(copy_horizontal_unlifted)
copy_vertical = lift(copy_vertical_unlifted)
crop_inside = lift(crop_inside_unlifted)
expand_twice = lift(expand_twice_unlifted)
expand_three = lift(expand_three_unlifted)
expand_four = lift(expand_four_unlifted)
horizontal_combi_twice = lift(horizontal_combi_twice_unlifted)
horizontal_combi_three = lift(horizontal_combi_three_unlifted)
vertical_combi_twice = lift(vertical_combi_twice_unlifted)
vertical_combi_three = lift(vertical_combi_three_unlifted)
kronecker_expansion = lift(kronecker_expansion_unlifted)
expansion_fliplr = lift(expansion_fliplr_unlifted)
expansion_flipud = lift(expansion_flipud_unlifted)
overlapV = lift(overlapV_unlifted)
overlapH = lift(overlapH_unlifted)
three_diagonal_pattern = lift(three_diagonal_pattern_unlifted)
expand_by_unique_colors = lift(expand_by_unique_colors_unlifted)
kronecker_negative_expansion = lift(kronecker_negative_expansion_unlifted)
kronecker_nn_expansion = lift(kronecker_nn_expansion_unlifted)
kronecker_minimum_position = lift(kronecker_minimum_position_unlifted)
kronecker_maximum_position = lift(kronecker_maximum_position_unlifted)
crop_from_left = lift(crop_from_left_unlifted)
crop_from_top = lift(crop_from_top_unlifted)
split_quarter = lift(split_quarter_unlifted)
expansion_negative_fliplr = lift(expansion_negative_fliplr_unlifted)
expansion_negative_flipud = lift(expansion_negative_flipud_unlifted)
expansion_by_unique_num = lift(expansion_by_unique_num_unlifted)
clockwise_expansion = lift(clockwise_expansion_unlifted)
Hoverlap_check = lift(Hoverlap_check_unlifted)
Voverlap_check = lift(Voverlap_check_unlifted)
horizontal_combi_five = lift(horizontal_combi_five_unlifted)
expansion_8directions = lift(expansion_8directions_unlifted)
addition_by_size = lift(addition_by_size_unlifted)
pickup_objects = lift(pickup_objects_unlifted)
make_copy_in_right_side = lift(make_copy_in_right_side_unlifted)
find_separation_lines = lift(find_separation_lines_unlifted)
unique_center_to_edge = lift(unique_center_to_edge_unlifted)
closed_area_to_yellow = lift(closed_area_to_yellow_unlifted)
closed_area_to_orange = lift(closed_area_to_orange_unlifted)
closed_area_to_green = lift(closed_area_to_green_unlifted)
closed_area_to_red = lift(closed_area_to_red_unlifted)
closed_area_to_blue = lift(closed_area_to_blue_unlifted)
gray_hanten = lift(gray_hanten_unlifted)
gray_copy = lift(gray_copy_unlifted)
change_gray_area = lift(change_gray_area_unlifted)
vertical_combi_three_another = lift(vertical_combi_three_another_unifted)
new_formation = lift(new_formation_unifted)
new_formation2 = lift(new_formation2_unlifted)
kronecker_straight = lift(kronecker_straight_unlifted)
blue_red = lift(blue_red_unlifted)            
overlap_color_objects = lift(overlap_color_objects_unlifted)
new_expansion = lift(new_expansion_unlifted)

# Program evaluation


We define our building blocks for programs (the functions in our DSL). We will define a program as a list of functions from our DSL ; `program: [[np.array] -> [np.array]]`. The instructions in our programs will be executed *from left to right*. This mean that if we want to first `splitByColor` and then compute the `negative` of the image, we need to write `[splitByColor, negative]` in this order.

In [9]:
def program_desc(program):
    """ Create a human readable description of a program. """
    desc = [x.__name__ for x in program]
    return(' >> '.join(desc))

# Display the program description alongside its output
program = [splitH, groupByColor, negative, intersect]
print(program_desc(program))

splitH_lifted >> groupByColor_lifted >> negative_lifted >> intersect


In [10]:
def evaluate(program: [], input_image: np.array):
    # Make sure the input is a np.array
    input_image = np.array(input_image)
    assert type(input_image) == np.ndarray
    
    # Apply each function on the image
    image_list = [input_image]
    for fct in program:
        # Apply the function
        image_list = fct(image_list)
        # Filter out empty images
        image_list = [img for img in image_list if img.shape[0] > 0 and img.shape[1] > 0]
        # Break if there is no data
        if image_list == []:
            return []
    return image_list        

# Program generation (Genetic Algorithm)
We now have a simple and powerful language to express various transformation on images. But someone or something still have to write the actual program that can solve a task. In this part, we will implement a naive but somewhat efficient genetic algorithm that will be able to find by itself the solution to a task.

The strategy will be as follow:

* We generate random program with one node, and then run them. We keep the best solution (the *elites* of our population).
* Starting from this best solutions, we create new program though mutation. We avaluate them again and update our collection of elite.
* We continue doing this process again and again... until a solution is found.
---
Since we use multiple fitness function, our aproache can be qualified of [multi-objectives](https://en.wikipedia.org/wiki/Multi-objective_optimization) : we try to optimise multiple objectives at the same time. Our *elites* can be understood as an approximation of the pareto surface (collection of pareto optimal solution). In our specific case, when a solution to the task exists in our DSL, their exists a global minimum that will be smaller than any candidate. In a such case the pareto surface is reduced to a single point. Nethertheless, this is a good image to keep in mind to understand what the collection of *elites* represent.

# Fitness

To help our algorithm progress in the right direction, we need a way to give a score to an existing program. The smaller is the score of the program, the closer we are to the solution. One can think of this score as a distance of our program to the optimal solution.

Notice that one can think of this program as a minimization problem (minimize `score`) or maximization problem (minimize `-score`). On machine learning it is common to minimise a distance wereas in genetic algorithm literature you can read that we maximize the fitness of an agent^1. Both convention work perfectly, but it is more convenient if we choose one and stick to it. Therefore, we will MINIMIZE the score of our programs.

Because we can't really comme up with one single good score function that would describe well the progression of the algorithm on all task of the dataset, we will evaluate how our program perform on different aspects through a collection of them.

^1: The reason you see maximization and positive score in Genetic Programming literature is that you need all your values to be positive in order to build a probability distribution over your population. Since we use an elitist algorithm instead of a sampling of the population for reproduction, we do not need this restriction.

In [11]:
def are_two_images_equals(a, b):
    if tuple(a.shape) == tuple(b.shape):
        if (np.abs(b-a) < 1).all():
            return True
    return False

def is_all_solution(program, task, verbose=True):
    for sample in task: # For each pair input/output
        i = np.array(sample['input'])
        o = np.array(sample['output'])

        # Evaluate the program on the input
        images = evaluate(program, i)
        if len(images) < 1:
            return False
        
        # The solution should be in the 3 first outputs 
        images = images
        
        # Check if the output is in the images produced
        is_program_of_for_sample = any([are_two_images_equals(x, o) for x in images])
        if not is_program_of_for_sample:
            return False
    
    return True

def is_solution(program, task, verbose=True):
    for sample in task: # For each pair input/output
        i = np.array(sample['input'])
        o = np.array(sample['output'])

        # Evaluate the program on the input
        images = evaluate(program, i)
        if len(images) < 1:
            return False
        
        # The solution should be in the 3 first outputs 
        images = images[:3]
        
        # Check if the output is in the images produced
        is_program_of_for_sample = any([are_two_images_equals(x, o) for x in images])
        if not is_program_of_for_sample:
            return False
    
    return True

In [12]:
def width_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right width. Less is better."""
    return np.abs(predicted.shape[0] - expected_output.shape[0])

def height_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right height. Less is better."""
    return np.abs(predicted.shape[1] - expected_output.shape[1])

def activated_pixels_fitness(p, e):
    """ How close the predicted image to have the right pixels. Less is better."""
    shape = (max(p.shape[0], e.shape[0]), max(p.shape[1], e.shape[1]))
    diff = np.zeros(shape, dtype=int)
    diff[0:p.shape[0], 0:p.shape[1]] = (p > 0).astype(int)
    diff[0:e.shape[0], 0:e.shape[1]] -= (e > 0).astype(int)
    
    return (diff != 0).sum()

def colors_fitness(p, e):
    p_colors = np.unique(p)
    e_colors = np.unique(e)
    
    nb_inter = len(np.intersect1d(p_colors, e_colors))

    return (len(p_colors) - nb_inter) + (len(e_colors) - nb_inter)

def colors_num_sum_fitness(p, e):
    p_h, p_w = p.shape
    e_h, e_w = e.shape
    p_colors_sum = np.sum(np.sum(p))
    e_colors_sum = np.sum(np.sum(e))
    return np.abs(p_colors_sum - e_colors_sum)

fitness_functions = [height_fitness, width_fitness,  colors_fitness, colors_num_sum_fitness] #activated_pixels_fitness, 
coefficients = [1, 1, 1, 1] #, 1

In [13]:
# ([[np.array] -> [np.array]], Taks) -> (int, int, ..., int)
def evaluate_fitness(program, task, coefficients):
    """ Take a program and a task, and return its fitness score. """
    score = np.zeros((len(fitness_functions)))
    
    # For each sample
    for sample in task:
        i = np.array(sample['input'])
        o = np.array(sample['output'])
        
        # For each fitness function
        for index, fitness_function in enumerate(fitness_functions):
            images = evaluate(program, i)
            if images == []: # Penalize no prediction!
                score[index] += 500
            else: # Take only the score of the first output
                score[index] = fitness_function(images[0], o)
    total_score = sum([score[di] * coefficients[di] for di in range(len(score))])
    return total_score 

# GA main algorithm
Now that we can compare two programs we need a way to generate some of them. We will generate them randomly from a pool of best candidate.
For the initial run, and also to be able to evaluate fresh candidates, we will also allow spontaneous generation of new born one instruction programs.

In [14]:
def build_candidates(allowed_nodes=[identity], best_candidates=[], nb_candidates=400): 
    """
    Create a poll of fresh candidates using the `allowed_nodes`.
    The pool contain a mix of new single instructions programs
    and mutations of the best candidates.
    """
    new_candidates = []
    length_limit = 2 # Maximal length of a program
    
    def random_node():
        return random.choice(allowed_nodes)
    
    while(len(new_candidates) < nb_candidates): # Until we have enougth new candidates
        # Add 10 new programs
        for i in range(10):
            new_candidates += [[random_node()]]
        
        # Create new programs based on each best candidate
        for best_program in best_candidates:
            # Add one op on its right but limit the length of the program
            if len(best_program) < length_limit - 1:
                random_program= random_node()
                if random_program not in best_program:
                    new_candidates += [best_program + [random_program]]
            # Add one op on its left but limit the length of the program
            if len(best_program) < length_limit - 1:
                random_program = random_node()
                if random_program not in best_program:
                    new_candidates += [[random_program] + best_program]
            # Mutate one instruction of the existing program
            new_candidates += [list(best_program)]
            new_candidates[-1][random.randrange(0, len(best_program))] = random_node()
   
    # Truncate if we have too many candidates
    np.random.shuffle(new_candidates)
    return new_candidates[:nb_candidates]

In [15]:
def random_node(allowed_nodes):
    return random.choice(allowed_nodes)

def generate_init_dsl(dsls, nums): # generate initial species
    new_candidates = []
    while len(new_candidates) < nums:
        new_candidates += [[random_node()]]
    return new_candidates 

def generate_roulette(dsls, task): # generate score for roulette selection
    fitness_vec = [evaluate_fitness(i, task, coefficients) for i in dsls]
    total = np.sum([1/(i+1) for i in fitness_vec])
    roulette = np.zeros(len(fitness_vec))
    for i in range(len(fitness_vec)):
        roulette[i] = (1/(fitness_vec[i]+1)) / total
    return roulette

def roulette_choice(dsls, task): # select good dsls to keep
    roulette = generate_roulette(dsls, task)
    choiced_ind = np.random.choice(len(roulette), 2, replace = True, p = roulette)
    choiced_dsls = [dsls[i] for i in choiced_ind]
    return choiced_dsls

# https://stackoverflow.com/questions/16547820/crossover-different-length-genotypes
def simple_crossover(parent1, parent2): # crossover
    num = min(len(parent1), len(parent2))
    cross_point = random.randrange(1,num-1)
    child1_1 = parent1[:cross_point]
    child1_2 = parent1[cross_point:]
    child2_1 = parent2[:cross_point]
    child2_2 = parent2[cross_point:]
    return child1_1+child2_2, child2_1+child1_2
    
def mutation(programs, num_mutation, p_value, allowed_nodes): # mutation
    ans = []
    for i in range(num_mutation):
        mutation_flg = np.random.choice(2,1,p=[1-p_value, p_value])
        if mutation_flg == 1:
            mutated_program = random_node(programs)
            num = len(mutated_program)
            point = random.randrange(0,num-1)
            new_dsl = random_node(allowed_nodes)
            mutated_program[point] = new_dsl
            ans.append(mutated_program)
    return ans

In [16]:
def new_build_candidates(length_limit, allowed_nodes): #brute force search
    new_candidates = []
    index_list = permutations([i for i in range(len(allowed_nodes))],length_limit)
    
    for ind in index_list:
        new_program = [allowed_nodes[i] for i in ind]
        new_candidates += [new_program]
   
    return new_candidates

# Solve the task
This is the last step to our genetic algorithm. We have all the building blocks:
 * Generating both new programs and mutation of existing solutions
 * Evaluating the fitness score of a program
 * Comparing two programs to know if one perform better than the other
 * Detecting when a solution was found
 
We can now write a function that will keep generating programs with increasing complexity until a solution is found.
Using our partial order, we are going to keep the best candidates. Because the order is partial,
there is no bound on how many uncomparables candidates we may have at a given iteration.

In [17]:
def build_model(task, class_num, coefficients, flg, generations=5, verbose=True):
    #change candidate nodes by class num
    if class_num >= 8:
        candidates_nodes=[tail, union, intersect,sortByColor, sortByWeight, reverse, cropToContent, 
                            groupByColor, negative, pickup_objects, gray_hanten, 
                            rotation1,rotation2,rotation3, unrotation1,unrotation2,unrotation3, fliplr, flipud, fliptr, represent_only_inside,
                            find_element_with_smallest_noise_objects, find_element_with_largest_noise_objects,
                            find_horizontal_asymmetric_objects, find_horizontal_symmetric_objects,
                            find_vertical_asymmetric_objects, find_vertical_symmetric_objects, 
                            find_objects_with_largest_uniques, find_objects_with_smallest_uniques,
                            find_unique_objects, find_same_objects, three_diagonal_pattern, 
                            expansion_negative_fliplr, expansion_negative_flipud, expansion_by_unique_num,
                            Hoverlap_check, Voverlap_check, horizontal_combi_five, addition_by_size, expansion_8directions, clockwise_expansion,
                            combine_into_one_piece, combine_into_one_piece2, combine_hole_objects, splitV,copy_horizontal, copy_vertical, crop_inside,
                            kronecker_expansion, expansion_fliplr, expansion_flipud,overlapV, overlapH,
                            three_diagonal_pattern,expand_by_unique_colors, kronecker_negative_expansion, kronecker_minimum_position, kronecker_maximum_position,
                            crop_from_left, crop_from_top,split_quarter, splitH, vertical_combi_three_another, horizontal_combi_three, horizontal_combi_twice,    
                            new_formation, new_formation2, copy_and_chng_color, kronecker_straight, blue_red, overlap_color_objects, new_expansion]
    else:
        # np.array -> [np.array]
        candidates_nodes=[identity]
    
    best_candidates = {} # A dictionary of {score:candidate}
    for i in range(generations):
        candidates = build_candidates(candidates_nodes, best_candidates.values()) # Create a list of candidates
        #candidates = new_build_candidates(generations, candidates_nodes)
        for candidate in candidates:
            score = evaluate_fitness(candidate, task, coefficients)
            is_uncomparable = True # True if we cannot compare the two candidate's scores because of empty best_candidates
            
            best_candidates_items = list(best_candidates.items()) # Compare the new candidate to the existing best candidates    
            for best_score, best_candidate in best_candidates_items:
                if score <= best_score:
                    del best_candidates[best_score] # Remove previous best candidate and add the new one
                    best_candidates[score] = candidate
                    is_uncomparable = False
            if is_uncomparable:
                best_candidates[score] = candidate
        for program in best_candidates.values(): # For each best candidate, we look if we have an answer
            if is_solution(program, task):
                ans = program
                return ans
    return None

In [18]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

sample_sub = pd.read_csv(data_path/'sample_submission.csv')
sample_sub = sample_sub.set_index('output_id')

In [19]:
mode = "eval"
if mode=='eval':
    task_path = evaluation_path
    df = evaluation_descriptive_df
elif mode=='train':
    task_path = training_path
    df = training_descriptive_df
elif mode=='test':
    task_path = test_path
    df = test_descriptive_df
    
all_task_ids = sorted(os.listdir(task_path))
for i in range(len(all_task_ids)):
    flg = False
    task_file = str(task_path / all_task_ids[i])
    class_num = df[df.task_name==all_task_ids[i]]["class"].values[0]
    with open(task_file, 'r') as f:
        task = json.load(f)
        
    # tile and symmetry check --- 
    train_ans_for_tile_and_symmetry = []
    for j, t in enumerate(task["train"]):
        t_in, t_out = np.array(t["input"]).astype('uint8'), np.array(t["output"]).astype('uint8')
        t_pred, feat = call_pred_train(t_in, t_out, patch_image)
        train_ans_for_tile_and_symmetry.append(np.all(t_pred==t_out))
    # ---------------------------
    
    if np.all(train_ans_for_tile_and_symmetry): # tile and symmetry ------
        print("data no: " + str(i) + " class: " + str(class_num) + ", Found program: Tile and symmetry in "+str(task_file))
        if mode == 'test':
            for task_num in range(len(task["test"])):
                t_in = np.array(task["test"][task_num]["input"]).astype('uint8')
                t_pred = call_pred_test(t_in, patch_image, feat) 
                t_pred = flattener(t_pred.astype(int).tolist())
                sample_sub.loc[f'{all_task_ids[i][:-5]}_{task_num}','output'] = t_pred
                
    else: # DSL + GA -----------------------------------------------------
        if class_num >= 8:
            if is_all_solution([pickup_objects], task['train'], verbose=True):
                flg = True
            program = build_model(task['train'], class_num, coefficients, flg, verbose=True)
            if program is None:
                print("data no: " + str(i) + " class: " + str(class_num) + ", No program was found in "+str(task_file))
            else:
                print("data no: " + str(i) + " class: " + str(class_num) + ", Found program in "+str(task_file) + " " + str(program_desc(program)))
                if mode == 'test':
                    for task_num in range(len(task["test"])):
                        images = evaluate(program, np.array(task['test'][task_num]["input"]))
                        images = images[:3] # The solution should be in the 3 first outputs
                        #print(images)
                        preds_list = [flattener(image.tolist()) for image in images]
                        #print(' '.join(preds_list))
                        sample_sub.loc[f'{all_task_ids[i][:-5]}_{task_num}','output'] = ' '.join(preds_list)

data no: 0 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/00576224.json
data no: 6 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0692e18c.json kronecker_negative_expansion_lifted
data no: 9 class: 9, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0934a4d8.json
data no: 11 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0a1d4ef5.json
data no: 16 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0c786b71.json new_formation2_lifted
data no: 24 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/12997ef3.json
data no: 26 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/136b0064.json
data no: 33 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/

In [20]:
sample_sub.to_csv('submission.csv')
sample_sub.head()

,output
output_id,
00576224_0,|32|78| |32|78| |00|00|
009d5c81_0,|00000000000000|00000888888888|00000800080808|...
00dbd492_0,|00000000000222220000|02222222220200020000|020...
03560426_0,|0000000000|0000000000|0000000000|0000000000|0...
05a7bcf2_0,|000000000020000000080000000000|00000000002220...


- **clear data no: 0 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/00576224.json　**
- **clear data no: 6 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0692e18c.json kronecker_negative_expansion_lifted**
- **clear data no: 9 class: 9, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0934a4d8.json**
- data no: 11 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0a1d4ef5.json
- **clear data no: 16 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/0c786b71.json**
- **できるはず data no: 24 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/12997ef3.json**
- data no: 26 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/136b0064.json
- **できるはず data no: 33 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/15696249.json**
- もうちょっと点々  data no: 43 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/1a6449f1.json
- **data no: 48 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/1d0a4b61.json**
- **data no: 52 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/1e97544e.json**
- data no: 53 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2037f2c7.json
- **expand_twice > blue_red できるはず data no: 54 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2072aba6.json**
- **clear data no: 55 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/20818e16.json**
- **できるはず pickup_objects, new_expansiondata no: 64 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2697da3f.json**
- data no: 65 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2753e76c.json
- **data no: 67 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/27f8ce4f.json kronecker_maximum_position_lifted**
- data no: 73 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2c0b0aff.json
- data no: 75 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/2f0c5170.json
- data no: 76 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/310f3251.json
- data no: 90 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/3979b1a8.json
- data no: 95 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/3ee1011a.json
- data no: 98 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/414297c0.json
- **data no: 107 class: 1, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/47996f11.json**
- data no: 108 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/48131b3c.json
- data no: 109 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/4852f2fa.json
- **data no: 110 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/48f8583b.json kronecker_minimum_position_lifted**
- **data no: 111 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/4aab4007.json**
- data no: 120 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/505fff84.json
- data no: 123 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/50aad11f.json
- data no: 135 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/5833af48.json
- data no: 138 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/59341089.json
- **data no: 143 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/5b6cbef5.json kronecker_expansion_lifted**
- **data no: 148 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/60c09cac.json expand_by_unique_colors_lifted**
- data no: 158 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/67636eac.json
- **data no: 159 class: 7, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/67b4a34d.json**
- data no: 163 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/695367ec.json
- data no: 170 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/6f473927.json
- data no: 171 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/7039b2d7.json
- data no: 178 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/73ccf9c2.json
- **data no: 184 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/7953d61e.json clockwise_expansion_lifted**
- data no: 186 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/7bb29440.json
- data no: 188 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/7c9b52a0.json
- data no: 195 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/81c0276b.json
- data no: 196 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/833dafe3.json
- data no: 203 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/8719f442.json
- data no: 208 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/8b28cd80.json
- data no: 212 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/8e2edd66.json
- **data no: 216 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/903d1b4a.json**
- **data no: 219 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/929ab4e9.json**
- data no: 222 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/93b4f4b3.json
- data no: 224 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/94133066.json
- **data no: 232 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/981571dc.json**
- **data no: 251 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/a59b95c0.json expansion_by_unique_num_lifted**
data no: 252 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/a680ac02.json
data no: 258 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/aab50785.json
data no: 263 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ad7e01d0.json
data no: 265 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/aee291af.json
- data no: 266 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/af22c60d.json
data no: 268 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/b0722778.json
data no: 269 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/b0f4d537.json
data no: 274 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/b4a43f3b.json
data no: 275 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/b7999b51.json
- **data no: 285 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/bc4146bd.json horizontal_combi_five_lifted**
data no: 293 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c1990cce.json
data no: 294 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c3202e5a.json
- **data no: 296 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c48954c1.json expansion_8directions_lifted**
- data no: 298 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c64f1187.json
- data no: 299 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c658a4bd.json
- **data no: 300 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c663677b.json**
- data no: 305 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/c92b942c.json
- **data no: 308 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ca8f78db.json**
- data no: 309 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/cad67732.json
- **data no: 311 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ccd554ac.json addition_by_size_lifted**
data no: 312 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/cd3c21df.json
data no: 314 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ce8d95cc.json
data no: 317 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/d017b73f.json
- **data no: 325 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/d4b1c2b1.json expand_by_unique_colors_lifted**
data no: 326 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/d4c90558.json
- **data no: 327 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/d56f2372.json crop_from_left_lifted**
data no: 328 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/d5c634a2.json
data no: 336 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/de493100.json
data no: 340 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e1baa8a4.json
data no: 351 class: 8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e633a9e5.json
- **data no: 352 class: 9, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e66aafb8.json**
- data no: 355 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e6de6e8f.json
- data no: 359 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e78887d1.json
- data no: 360 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e7a25a18.json
- data no: 363 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e872b94a.json
- **data no: 365 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e95e3d8e.json**
- data no: 368 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/e9b4f6fc.json
- **data no: 371 class: 6, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ea959feb.json**
- **data no: 375 class: 8, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/ed98d772.json clockwise_expansion_lifted**
- data no: 377 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/f0afb749.json
- **data no: 383 class: 9, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/f4081712.json**
- **data no: 385 class: 9, Found program in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/f5aa3634.json pickup_objects_lifted**
- **data no: 387 class: 4, Found program: Tile and symmetry in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/f823c43c.json**
- data no: 393 class: 9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/evaluation/fb791726.json